In [220]:
%%capture
! gdown 18UD2EGKwq3YjVMBPsS8kVz0e671p7vzV
! gdown 1xAWZ_tEjEAFsosCdax78lrWkLnUgoWSn

import pandas as pd

covid_df = pd.read_csv("/content/COVID_county_data.csv")
census_df = pd.read_csv("/content/acs2017_census_tract_data.csv")

In [221]:
census_df.columns

Index(['TractId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment'],
      dtype='object')

In [222]:
census_df = census_df[['TractId', 'State', 'County', 'TotalPop', 'IncomePerCap', 'Poverty']]
census_df.head()

,TractId,State,County,TotalPop,IncomePerCap,Poverty
0,1001020100,Alabama,Autauga County,1845,33018.0,10.7
1,1001020200,Alabama,Autauga County,2172,18996.0,22.4
2,1001020300,Alabama,Autauga County,3385,21236.0,14.7
3,1001020400,Alabama,Autauga County,4267,28068.0,2.3
4,1001020500,Alabama,Autauga County,9965,36905.0,12.2


In [223]:
def process_acs_data(data):

    data = data.groupby(['State', 'County']).agg({
      'TotalPop': 'sum',
      'IncomePerCap': 'mean',
      'Poverty': 'mean'
    }).reset_index()
    data['ID'] = range(1, len(data) + 1)

    return data

In [224]:
county_info = process_acs_data(census_df)
county_info = county_info.reset_index()
county_info['County'] = county_info['County'].str.replace(' County', '')
county_info['County'] = county_info['County'].str.replace(' Parish', '')
county_info.head()

,index,State,County,TotalPop,IncomePerCap,Poverty,ID
0,0,Alabama,Autauga,55036,26588.166667,14.558333,1
1,1,Alabama,Baldwin,203360,29130.709677,12.874194,2
2,2,Alabama,Barbour,26201,17891.666667,27.755556,3
3,3,Alabama,Bibb,22580,21799.000000,13.925000,4
4,4,Alabama,Blount,57667,21598.444444,16.422222,5


In [225]:
def get_pop_pov_pci(data, county, state):
    return data[(data['County'] == county) & (data['State'] == state)]

In [226]:
loudan = get_pop_pov_pci(county_info, 'Loudoun', 'Virginia')
loudan

,index,State,County,TotalPop,IncomePerCap,Poverty,ID
2968,2968,Virginia,Loudoun,374558,50391.015625,3.884375,2969


In [227]:
washington = get_pop_pov_pci(county_info, 'Washington', 'Oregon')
washington

,index,State,County,TotalPop,IncomePerCap,Poverty,ID
2241,2241,Oregon,Washington,572071,34970.817308,10.446154,2242


In [228]:
harlan = get_pop_pov_pci(county_info, 'Harlan', 'Kentucky')
harlan

,index,State,County,TotalPop,IncomePerCap,Poverty,ID
1040,1040,Kentucky,Harlan,27548,16010.363636,33.318182,1041


In [229]:
malheur = get_pop_pov_pci(county_info, 'Malheur', 'Oregon')
malheur

,index,State,County,TotalPop,IncomePerCap,Poverty,ID
2230,2230,Oregon,Malheur,30421,17966.428571,24.414286,2231


In [230]:
most_populous_county = county_info[county_info['TotalPop'] == county_info['TotalPop'].max()]
least_populous_county = county_info[county_info['TotalPop'] == county_info['TotalPop'].min()]

In [231]:
print("Most populous county in the USA:\n", most_populous_county)
print("Least populous county in the USA:\n", least_populous_county)

Most populous county in the USA:
      index       State       County  TotalPop  IncomePerCap    Poverty   ID
204    204  California  Los Angeles  10105722  31389.413867  17.323803  205
Least populous county in the USA:
       index  State  County  TotalPop  IncomePerCap  Poverty    ID
2751   2751  Texas  Loving        74       35530.0     17.1  2752


In [232]:
covid_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [233]:
covid_df['date'] = pd.to_datetime(covid_df['date'])
covid_df['month'] = covid_df['date'].dt.to_period('M')
COVID_monthly = covid_df.groupby(['month', 'county', 'state', 'fips']).agg({
    'cases': 'sum',
    'deaths': 'sum'
}).reset_index()

In [234]:
COVID_monthly = pd.merge(COVID_monthly, county_info, how='left', left_on=['county', 'state'], right_on=['County', 'State'])

COVID_monthly = COVID_monthly[['month', 'county', 'state', 'fips', 'cases', 'deaths', 'ID']]
COVID_monthly['ID'] = COVID_monthly['ID'].astype('Int64')
COVID_monthly = COVID_monthly.dropna()
COVID_monthly.head()

,month,county,state,fips,cases,deaths,ID
0,2020-01,Cook,Illinois,17031.0,10,0.0,611
1,2020-01,Los Angeles,California,6037.0,6,0.0,205
2,2020-01,Maricopa,Arizona,4013.0,6,0.0,104
3,2020-01,Orange,California,6059.0,7,0.0,216
4,2020-01,Santa Clara,California,6085.0,1,0.0,229


In [235]:
COVID_monthly[(COVID_monthly['county'] == 'Malheur') &
 ((COVID_monthly['month'] == '2020-08') |
  (COVID_monthly['month'] == '2021-01') |
   (COVID_monthly['month'] == '2021-02'))]


,month,county,state,fips,cases,deaths,ID
16139,2020-08,Malheur,Oregon,41045.0,28163,459.0,2231
32214,2021-01,Malheur,Oregon,41045.0,96297,1627.0,2231
35432,2021-02,Malheur,Oregon,41045.0,65951,1137.0,2231


In [236]:
total_covid_data = COVID_monthly.groupby('ID').agg({
    'cases': 'sum',
    'deaths': 'sum'
})

COVID_summary = pd.merge(county_info, total_covid_data, how='left', left_on='ID', right_on='ID')
COVID_summary['TotalCasesPer100K'] = (COVID_summary['cases'] / COVID_summary['TotalPop']) * 100000
COVID_summary['TotalDeathsPer100K'] = (COVID_summary['deaths'] / COVID_summary['TotalPop']) * 100000

COVID_summary = COVID_summary[['ID', 'County', 'State', 'TotalPop', 'Poverty', 'IncomePerCap', 'cases', 'deaths', 'TotalCasesPer100K', 'TotalDeathsPer100K']]
COVID_summary.rename(columns={
    'IncomePerCap': 'PerCapitaIncome',
    'cases': 'TotalCases',
    'deaths': 'TotalDeaths'
})

COVID_summary.head()

,ID,County,State,TotalPop,Poverty,IncomePerCap,cases,deaths,TotalCasesPer100K,TotalDeathsPer100K
0,1,Autauga,Alabama,55036,14.558333,26588.166667,645935.0,9042.0,1.173659e+06,16429.246312
1,2,Baldwin,Alabama,203360,12.874194,29130.709677,2003567.0,23041.0,9.852316e+05,11330.153423
2,3,Barbour,Alabama,26201,27.755556,17891.666667,268771.0,4077.0,1.025804e+06,15560.474791
3,4,Bibb,Alabama,22580,13.925000,21799.000000,261043.0,5272.0,1.156081e+06,23348.095660
4,5,Blount,Alabama,57667,16.422222,21598.444444,630106.0,8669.0,1.092663e+06,15032.861082


In [237]:
def get_pov_tcp100(county, state):
  return COVID_summary[(COVID_summary['County'] == county) & (COVID_summary['State'] == state)]

In [238]:
get_pov_tcp100('Washington', 'Oregon')

,ID,County,State,TotalPop,Poverty,IncomePerCap,cases,deaths,TotalCasesPer100K,TotalDeathsPer100K
2241,2242,Washington,Oregon,572071,10.446154,34970.817308,2157339.0,22455.0,377110.358679,3925.21208


In [239]:
get_pov_tcp100('Malheur', 'Oregon')

,ID,County,State,TotalPop,Poverty,IncomePerCap,cases,deaths,TotalCasesPer100K,TotalDeathsPer100K
2230,2231,Malheur,Oregon,30421,24.414286,17966.428571,453634.0,7770.0,1.491187e+06,25541.566681


In [240]:
get_pov_tcp100('Loudoun', 'Virginia')

,ID,County,State,TotalPop,Poverty,IncomePerCap,cases,deaths,TotalCasesPer100K,TotalDeathsPer100K
2968,2969,Loudoun,Virginia,374558,3.884375,50391.015625,2496450.0,35820.0,666505.58792,9563.27191


In [241]:
get_pov_tcp100('Harlan', 'Kentucky')

,ID,County,State,TotalPop,Poverty,IncomePerCap,cases,deaths,TotalCasesPer100K,TotalDeathsPer100K
1040,1041,Harlan,Kentucky,27548,33.318182,16010.363636,205984.0,3994.0,747727.60273,14498.330187
